<a href="https://colab.research.google.com/github/Sallb679/SafeGraph-Analysis/blob/main/Econ433_proj4_Boubacar_Sall_Cracker_Barrel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tabulate import tabulate

For Reference:
Benchmark is x (cracker barrel)
focal is y (chilis and grill)


Part 1: Prepare data



Question 1a (1 point): Use MergedSummary you have created at the end of Project 3. Display a table showing the number of observations for each state in this data for your own chain and your benchmark chain respectively. It is ok to use multiple tables if that is more convenient for you.




In [ ]:
# load this data and save it to a variable MergedSummary
file = '/content/drive/MyDrive/ECON_433_Project_3_data/MergedSummary.csv'
pd.set_option('display.max_columns', None)
MergedSummary = pd.read_csv(file)



In [ ]:
MergedSummary.head()

,region,date,dayofweek_x,Nunits_x,SumDailyVisits_x,MaxDailyVisitsPerUnit_x,MinDailyVisitsPerUnit_x,MedDailyVisitsPerUnit_x,dayofweek_y,Nunits_y,SumDailyVisits_y,MaxDailyVisitsPerUnit_y,MinDailyVisitsPerUnit_y,MedDailyVisitsPerUnit_y,BlueState
0,AL,2018-01-01,0,4,184,52,37,47.5,0,1,7,7,7,7.0,0
1,AL,2018-01-02,1,4,48,15,9,12.0,1,1,10,10,10,10.0,0
2,AL,2018-01-03,2,4,130,38,28,32.0,2,1,18,18,18,18.0,0
3,AL,2018-01-04,3,4,127,35,29,31.5,3,1,14,14,14,14.0,0
4,AL,2018-01-05,4,4,171,56,33,41.0,4,1,30,30,30,30.0,0


In [ ]:
# Count observations for each state (region) for the benchmark chain (Nunits_x)
benchmark_state_counts = MergedSummary.groupby('region')['Nunits_x'].count().reset_index()
benchmark_state_counts.columns = ['State', 'Benchmark Observations']

# Count observations for each state (region) for the focal chain (Nunits_y)
focal_state_counts = MergedSummary.groupby('region')['Nunits_y'].count().reset_index()
focal_state_counts.columns = ['State', 'Focal Observations']

# Display the tables
print("Number of Observations for Each State in the Benchmark Chain:")
print(benchmark_state_counts)

print("\nNumber of Observations for Each State in the Focal Chain:")
print(focal_state_counts)

Number of Observations for Each State in the Benchmark Chain:
   State  Benchmark Observations
0     AL                    1827
1     AR                    1827
2     AZ                    1827
3     CA                    1827
4     CO                    1827
5     FL                    1827
6     GA                    1827
7     IL                    1827
8     IN                    1827
9     KS                    1827
10    KY                    1827
11    MA                    1827
12    MD                    1827
13    MI                    1827
14    MO                    1827
15    MS                    1827
16    NC                    1827
17    NH                    1827
18    NJ                    1827
19    NY                    1827
20    OH                    1827
21    OK                    1827
22    PA                    1827
23    TN                    1827
24    TX                    1827
25    VA                    1827
26    WI                    1827
27    WV      

Question 1b (2 points): Download the data for Census demographics. Merge your MergedSummary data with the demographic data by the variable "State".

Answer the questions:

How many observations do you have in MergedSummary?
How many observations do you have in the Census demographic data?
How many observations are matched between the two after the merge?
How many observations are in MergedSummary but not the demographic data?
How many observations are in the demographic data but not in MergedSummary?
What explains the imperfect match?
Keep only the matched observation for the rest of the project, use the resulting dataset as the final analysis sample.





In [ ]:
#load this data save is as Census_2016
file2 = '/content/drive/MyDrive/ECON_433_Project_3_data/census_state_2016.csv'
pd.set_option('display.max_columns', None)
Census_2016 = pd.read_csv(file2)
Census_2016.head()

,state,total_population,male,female,median_age,age_over18,age_over21,age_over62,age_over65,single_race,two_or_more_race,white,black,indian_alaska,asian,hawaiian_pacific,other_races,hispanic_latino,housing_units,citizen,citizen_male,citizen_female,state_abbr
0,Alabama,4863300,2356052,2507248,39.0,3763783,3553484,964274,783952,4779442,83858,3389735,1338854,56661,85704,3769,77234,199686,2230180,3670229,1742667,1927562,AL
1,Alaska,741894,390573,351321,33.5,554598,526655,99799,75832,677933,63961,536380,37479,145982,61637,15903,15826,51304,310672,531845,280943,250902,AK
2,Arizona,6931071,3444228,3486843,37.5,5299649,5003462,1411716,1169826,6681391,249680,5465822,384932,392975,289425,31883,643100,2144775,2961136,4801062,2357954,2443108,AZ
3,Arkansas,2988248,1468782,1519466,38.0,2280888,2142654,594353,485630,2910827,77421,2358554,496098,53548,53699,11608,97373,214544,1354801,2193217,1061614,1131603,AR
4,California,39250017,19491231,19758786,36.4,30156768,28534107,6597908,5348439,37370367,1879650,24980439,2758577,746753,6432756,319337,6074661,15280776,14061375,25197798,12339669,12858129,CA


In [ ]:
# Count initial observations in each dataset
mergedsummary_obs = MergedSummary.shape[0]
census_obs = Census_2016.shape[0]

# Merge the data on 'region' from MergedSummary and 'state_abbr' from Census_2016
merged_data = pd.merge(MergedSummary, Census_2016, left_on='region', right_on='state_abbr', how='inner')
matched_obs = merged_data.shape[0]

# Left join to find observations in MergedSummary not in Census_2016
unmatched_in_mergedsummary = MergedSummary.merge(Census_2016, left_on='region', right_on='state_abbr', how='left', indicator=True)
unmatched_mergedsummary_count = unmatched_in_mergedsummary[unmatched_in_mergedsummary['_merge'] == 'left_only'].shape[0]

# Right join to find observations in Census_2016 not in MergedSummary
unmatched_in_census = Census_2016.merge(MergedSummary, left_on='state_abbr', right_on='region', how='left', indicator=True)
unmatched_census_count = unmatched_in_census[unmatched_in_census['_merge'] == 'left_only'].shape[0]

# Display counts and analysis results
print(f"Number of observations in MergedSummary: {mergedsummary_obs}")
print(f"Number of observations in Census demographic data: {census_obs}")
print(f"Number of matched observations: {matched_obs}")
print(f"Observations in MergedSummary but not in Census demographic data: {unmatched_mergedsummary_count}")
print(f"Observations in Census demographic data but not in MergedSummary: {unmatched_census_count}")

# For the final analysis, keep only matched observations
final_data = merged_data



Number of observations in MergedSummary: 51156
Number of observations in Census demographic data: 52
Number of matched observations: 51156
Observations in MergedSummary but not in Census demographic data: 0
Observations in Census demographic data but not in MergedSummary: 24


Explanation for Imperfect Match
When merging the MergedSummary dataset with the Census_2016 dataset, we notice that there are 28 states present in MergedSummary, whereas the Census_2016 dataset contains data for all 52 "states". As a result, there are 24 unmatched observations in the Census_2016 dataset for the states that do not appear in MergedSummary which add up to 52 and that makes sense.

Question 1c (2 points): Now using the final analysis sample, define your dependent variable as

                      Y = [Ln(SumDailyVisits+1) of your focal chain - Ln(SumDailyVisits+1) of your benchmark chain].

If you believe you can have a better variable describing the performance of your focal chain relative to your benchmark chain, you are more than welcome to use your own dependent variable, as long as you explain why your choice is better.

Define the list of your potential right-hand variables, which should include day of week indicators (Monday, Tuesday …, Sunday), some demographics, some state attributes (e.g. whether the state is red or blue as you have defined in Project 3). Feel free to add new variables to this list, as long as you articulate a reason for their relevance.

Summarize all these variables in a table with the number of observations, mean, median, standard deviation, minimum and maximum. It is ok to use multiple tables if that is more convenient for you. If some variables are categorical and thus cannot appear in this summary table, create separate table(s) to describe the frequency of each value in such categorical variable(s).



In [ ]:
# Create the dependent variable Y as the difference in log-transformed daily visits
final_data['Y'] = np.log(final_data['SumDailyVisits_x'] + 1) - np.log(final_data['SumDailyVisits_y'] + 1)

# Display the first few rows of the final data with the new dependent variable
final_data[['region', 'date', 'SumDailyVisits_x', 'SumDailyVisits_y', 'Y']].head()


,region,date,SumDailyVisits_x,SumDailyVisits_y,Y
0,AL,2018-01-01,184,7,3.140914
1,AL,2018-01-02,48,10,1.493925
2,AL,2018-01-03,130,18,1.930758
3,AL,2018-01-04,127,14,2.143980
4,AL,2018-01-05,171,30,1.713507


In [ ]:
# Select relevant continuous variables
continuous_vars = ['total_population', 'male', 'female', 'median_age',
                   'age_over18', 'age_over21', 'age_over65',
                   'white', 'black', 'asian', 'hispanic_latino', 'housing_units',
                   'citizen', 'citizen_male', 'citizen_female']

# Summary statistics for continuous variables
summary_continuous = final_data[continuous_vars].describe().T
summary_continuous.drop(columns='count', inplace=True)  # Avoid duplicating 'count' and 'Observations'
summary_continuous.rename(columns={'mean': 'Mean', '50%': 'Median', 'std': 'Std Dev', 'min': 'Min', 'max': 'Max'}, inplace=True)
summary_continuous['Observations'] = final_data[continuous_vars].count()

# Categorical variables and their frequency
categorical_vars = ['BlueState', 'dayofweek_x', 'dayofweek_y']
summary_categorical = {var: final_data[var].value_counts() for var in categorical_vars}

# Format and display continuous variables summary
print("Summary of Continuous Variables:")
print(tabulate(summary_continuous[['Observations', 'Mean', 'Median', 'Std Dev', 'Min', 'Max']],
               headers="keys", tablefmt="grid"))

# Format and display categorical variables summary
print("\nSummary of Categorical Variables:")
for var, value_counts in summary_categorical.items():
    print(f"\n{var} Summary:")
    print(tabulate(value_counts.reset_index(), headers=["Category", "Count"], tablefmt="grid"))


Summary of Continuous Variables:
+------------------+----------------+------------------+------------------+-------------+------------------+--------------+
|                  |   Observations |             Mean |           Median |     Std Dev |              Min |          Max |
+==================+================+==================+==================+=============+==================+==============+
| total_population |          51156 |      9.5765e+06  |      6.73149e+06 | 8.22645e+06 |      1.3348e+06  |  3.925e+07   |
+------------------+----------------+------------------+------------------+-------------+------------------+--------------+
| male             |          51156 |      4.70727e+06 |      3.28658e+06 | 4.07062e+06 | 664722           |  1.94912e+07 |
+------------------+----------------+------------------+------------------+-------------+------------------+--------------+
| female           |          51156 |      4.86923e+06 |      3.44549e+06 | 4.1565e+06  | 670073   

The values I included are continuous_vars = ['total_population', 'male', 'female', 'median_age',
                   'age_over18', 'age_over21', 'age_over65',
                   'white', 'black', 'asian', 'hispanic_latino', 'housing_units',
                   'citizen', 'citizen_male', 'citizen_female']
categorical_vars = ['BlueState', 'dayofweek_x', 'dayofweek_y']
I will not use all of these in the regression and Model but they may have explanatory power for the dailyvisits relationship between x and y


Part 2: Linear regression, LASSO and Ridge



Question 2a (2 points): Choose a list of variables to predict your Y. The list you choose could be a subset of the potential right-hand variables you have identified in Question 1c. I will refer to these variables as “features”.



Use the Python command “train_test_split()” to separate your final analysis sample into a training sample and a test sample.



Use function MinMaxScalar() to standardize all the “features” you choose, in both the training and test samples.



Answer the questions:

How many observations are there in the training sample?
How many in the test sample?
Choose at least two variables to comment on the difference between the two samples. For example, are they different in the months covered? Are they different in certain demographics? Feel free to use other variables for this comparison.




In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Define your features and dependent variable Y
features = ['Nunits_x', 'SumDailyVisits_x', 'MedDailyVisitsPerUnit_x',
    'Nunits_y', 'SumDailyVisits_y', 'MedDailyVisitsPerUnit_y',
    'total_population', 'median_age', 'BlueState', ]
target = 'Y'

# Split data into training and test samples
train_data, test_data = train_test_split(final_data, test_size=0.3, random_state=42)

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the scaler on training data, then apply it to test data
train_data_scaled = train_data.copy()
test_data_scaled = test_data.copy()
train_data_scaled[features] = scaler.fit_transform(train_data[features])
test_data_scaled[features] = scaler.transform(test_data[features])

# Check the number of observations
n_train = len(train_data_scaled)
n_test = len(test_data_scaled)

# Summarize key differences (example: demographics and month coverage)
comparison_summary = pd.DataFrame({
    "Feature": features,
    "Training Sample Mean": train_data_scaled[features].mean().values,
    "Test Sample Mean": test_data_scaled[features].mean().values
})

# Output the results
print(f"Number of observations in the training sample: {n_train}")
print(f"Number of observations in the test sample: {n_test}\n")

print("Comparison of Features Between Training and Test Samples:")
print(tabulate(comparison_summary, headers="keys", tablefmt="grid"))


Number of observations in the training sample: 35809
Number of observations in the test sample: 15347

Comparison of Features Between Training and Test Samples:
+----+-------------------------+------------------------+--------------------+
|    | Feature                 |   Training Sample Mean |   Test Sample Mean |
+====+=========================+========================+====================+
|  0 | Nunits_x                |              0.184315  |          0.182807  |
+----+-------------------------+------------------------+--------------------+
|  1 | SumDailyVisits_x        |              0.0945682 |          0.094227  |
+----+-------------------------+------------------------+--------------------+
|  2 | MedDailyVisitsPerUnit_x |              0.226404  |          0.227204  |
+----+-------------------------+------------------------+--------------------+
|  3 | Nunits_y                |              0.134413  |          0.136655  |
+----+-------------------------+-----------------

1. How many observations are there in the training and test samples?

Training sample: 35,809 observations
Test sample: 15,347 observations
2. Choose at least two variables to compare the training and test samples.

Comparison Between Training and Test Samples
Variable 1: BlueState
Training sample mean: 0.318216
Test sample mean: 0.328924
Comment: The proportion of observations from blue states is slightly higher in the test sample (32.89%) than in the training sample (31.82%). This indicates a small demographic difference in the political alignment distribution between the two datasets.
Variable 2: total_population
Training sample mean: 0.216538
Test sample mean: 0.219319
Comment: The average standardized total population is marginally higher in the test sample than in the training sample. While the difference is small, it suggests a slight variation in population distributions between the two samples.


Question 2b (2 points): Using Y and the standardized features in Question 2a, run a linear regression in your training sample. Report the coefficient estimates and R-squares.



What “features” have statistically significant coefficients (with p-value<0.05)? Among these significant “features”, which has a positive coefficient and which has a negative coefficient? Comment on whether their signs and magnitude confirm your prior.



If you extend the model to predict Y in the test sample, what is the R-squares in the test sample?



In [ ]:
import statsmodels.api as sm


# Add a constant (intercept) to the features
X_train = sm.add_constant(train_data_scaled[features])
y_train = train_data_scaled[target]

# Run the linear regression
model = sm.OLS(y_train, X_train).fit()

# Get the summary of the model
print("Linear Regression Results (Training Sample):")
print(model.summary())

# Get the R-squared value for the training sample
r_squared_train = model.rsquared
print(f"\nR-squared for training sample: {r_squared_train}")

# Now, for the test sample, we will also add a constant and scale the features
X_test = sm.add_constant(test_data_scaled[features])
y_test = test_data_scaled[target]

# Predict using the trained model
y_pred_test = model.predict(X_test)

# Calculate R-squared for the test sample
ss_total = np.sum((y_test - np.mean(y_test)) ** 2)
ss_residual = np.sum((y_test - y_pred_test) ** 2)
r_squared_test = 1 - (ss_residual / ss_total)
print(f"\nR-squared for test sample: {r_squared_test}")



Linear Regression Results (Training Sample):
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     9280.
Date:                Wed, 20 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:39:36   Log-Likelihood:                -37245.
No. Observations:               35809   AIC:                         7.451e+04
Df Residuals:                   35799   BIC:                         7.460e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

Linear Regression Results
Statistically Significant Features (p-value < 0.05):
All features in the model are statistically significant.

Positive Coefficients:
These features increase
𝑌
Y as their values rise:

Nunits_x:
4.52

MedDailyVisitsPerUnit_x:
4.10

median_age:
1.05

BlueState:
0.14

Negative Coefficients:
These features decrease
𝑌
Y as their values rise:

SumDailyVisits_x:
−
1.81

Nunits_y:
−
1.75

SumDailyVisits_y:
−
3.63

MedDailyVisitsPerUnit_y:
−
3.05

total_population:
−
0.48

R-Squared Values:
Training Sample R-Squared:
0.70

Test Sample R-Squared:
0.70

The close values for R-squared indicate the model fits the data well and generalizes effectively to the test sample.

Interpretation of Results
Positive Features:

Nunits_x and MedDailyVisitsPerUnit_x have the strongest positive impact on
𝑌
Y, suggesting that more units and higher median visits per unit drive the target variable up.
median_age positively correlates with
𝑌
Y, potentially reflecting stability in older populations.
Negative Features:

SumDailyVisits_y and MedDailyVisitsPerUnit_y reduce
𝑌
Y, possibly due to inefficiencies in these metrics.
The negative effect of total_population might indicate saturation in highly populated areas.
Overall, the model explains 70% of the variability in
𝑌
Y, making it a reliable predictor.



Question 2c (3 points): Using Y and the standardized features in Question 2a, run LASSO in your training sample.



Try at least three different values for the penalty parameter. For each value you have tried,

report the coefficient estimates and R-squares;
extend the model to predict Y in the test sample, and report the R-squares in the test sample.
Among the penalty parameters you have tried, which one is most appropriate? Explain how you arrive at this choice.



In the LASSO model with your preferred choice of the penalty parameter,

what “features” have zero (or very close to zero) coefficients?
What and how many “features” have non-zero coefficients?
Comment on whether their signs and magnitude confirm your prior.
What is the R-squares when you apply this model to the test sample? Is it higher or lower than what you find in Question 2b for linear regression?


Use the random permutation method to identify the five most important features in the above LASSO model.

What are they?
Comment on whether the order of their importance is consistent with your prior.


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

# Define features and target
features = ['Nunits_x', 'SumDailyVisits_x', 'MedDailyVisitsPerUnit_x',
            'Nunits_y', 'SumDailyVisits_y', 'MedDailyVisitsPerUnit_y',
            'total_population', 'median_age', 'BlueState']
target = 'Y'

X_train = train_data_scaled[features]
y_train = train_data_scaled[target]
X_test = test_data_scaled[features]
y_test = test_data_scaled[target]

# Penalty parameters to try
alphas = [0.01, 0.1, 1.0]

# Dictionary to store results
results = {}

# Run LASSO for different penalty parameters
for alpha in alphas:
    lasso = Lasso(alpha=alpha, random_state=42)
    lasso.fit(X_train, y_train)

    # Coefficients
    coefficients = pd.DataFrame({
        "Feature": features,
        "Coefficient": lasso.coef_
    })

    # R-squared for training and test samples
    r2_train = lasso.score(X_train, y_train)
    r2_test = lasso.score(X_test, y_test)

    # Store results
    results[alpha] = {
        "Model": lasso,
        "Coefficients": coefficients,
        "R2_Train": r2_train,
        "R2_Test": r2_test
    }

    # Print results for this penalty parameter
    print(f"\nResults for LASSO with alpha = {alpha}:")
    print(f"R-squared (Training): {r2_train}")
    print(f"R-squared (Test): {r2_test}")
    print("Coefficients:")
    print(coefficients.to_string(index=False))

# Choose the best penalty parameter (alpha) based on test R-squared
best_alpha = max(results, key=lambda a: results[a]['R2_Test'])
best_model = results[best_alpha]["Model"]

print(f"\nBest penalty parameter (alpha): {best_alpha}")
print(f"Test R-squared for best model: {results[best_alpha]['R2_Test']}")

# Features with non-zero coefficients
non_zero_coeffs = results[best_alpha]["Coefficients"].query("Coefficient != 0")
print(f"\nNumber of non-zero coefficients: {len(non_zero_coeffs)}")
print(non_zero_coeffs)

# Apply random permutation method for feature importance
perm_importance = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42)
importance_df = pd.DataFrame({
    "Feature": features,
    "Importance": perm_importance.importances_mean
}).sort_values(by="Importance", ascending=False)

# Five most important features
top_5_features = importance_df.head(5)
print("\nTop 5 Most Important Features (Random Permutation Method):")
print(top_5_features.to_string(index=False))




Results for LASSO with alpha = 0.01:
R-squared (Training): 0.6643322589258114
R-squared (Test): 0.6668577681513799
Coefficients:
                Feature  Coefficient
               Nunits_x     3.444635
       SumDailyVisits_x     0.000000
MedDailyVisitsPerUnit_x     2.367174
               Nunits_y    -1.742117
       SumDailyVisits_y    -2.784382
MedDailyVisitsPerUnit_y    -0.844260
       total_population    -0.460861
             median_age     0.754685
              BlueState     0.047932

Results for LASSO with alpha = 0.1:
R-squared (Training): 0.09196038714847621
R-squared (Test): 0.092499537788448
Coefficients:
                Feature  Coefficient
               Nunits_x     0.600674
       SumDailyVisits_x     0.000000
MedDailyVisitsPerUnit_x     0.000000
               Nunits_y    -0.000000
       SumDailyVisits_y    -0.000000
MedDailyVisitsPerUnit_y    -0.000000
       total_population    -0.000000
             median_age     0.000000
              BlueState    -0.013990



The most appropriate penalty parameter is 0.01.

Reason: At α = 0.01, the model achieves a balance between feature selection and predictive power, with an R-squared of 0.667 on the test sample, significantly better than α = 0.1 or 1.0.
Non-Zero Coefficients (α = 0.01):
The model retains 8 features with non-zero coefficients:

Nunits_x (3.44)
MedDailyVisitsPerUnit_x (2.37)
Nunits_y (-1.74)
SumDailyVisits_y (-2.78)
MedDailyVisitsPerUnit_y (-0.84)
total_population (-0.46)
median_age (0.75)
BlueState (0.05)
Zero Coefficients: SumDailyVisits_x was shrunk to zero, indicating it is not important in this model.
R-Squared Comparison:
LASSO (α = 0.01): Test R-squared = 0.667
Linear Regression: Test R-squared = 0.698
The LASSO model has a slightly lower R-squared compared to linear regression, but it improves interpretability by excluding less important features.

Feature Importance (Random Permutation Method):
The five most important features are:

Nunits_x (0.91)
Nunits_y (0.17)
MedDailyVisitsPerUnit_x (0.15)
SumDailyVisits_y (0.15)
median_age (0.05)
Interpretation:
Nunits_x is the most important predictor, consistent with expectations based on its large positive coefficient.
The importance of Nunits_y, MedDailyVisitsPerUnit_x, and SumDailyVisits_y aligns with their significant non-zero coefficients.
The low importance of median_age suggests it contributes less despite a positive coefficient.
Conclusion:
The LASSO model at α = 0.01 effectively balances predictive accuracy and feature selection. It identifies Nunits_x as the most impactful feature, confirming prior expectations about its significance. However, its slightly lower R-squared compared to linear regression reflects a trade-off for increased model simplicity.


Question 2d (3 points): Using Y and the standardized features in Question 2a, run Ridge in your training sample.



Try at least three different values for the penalty parameter. For each value you have tried,

report the coefficient estimates and R-squares;
extend the model to predict Y in the test sample, and report the R-squares in the test sample.


Among the penalty parameters you have tried, which one is most appropriate? Explain how you arrive at this choice.



In the Ridge model with your preferred choice of the penalty parameter,

what “features” have zero (or very close to zero) coefficients?
What and how many “features” have non-zero coefficients?
Comment on whether their signs and magnitude confirm your prior.
What is the R-squares when you apply this model to the test sample? Is it higher or lower than what you find in question 3b for linear regression and question 3c for LASSO?


Use the random permutation method to identify the five most important features in the above Ridge model. What are they? Comment on whether the order of their importance is consistent with your prior. Does Ridge identify the same five most important features as LASSO?

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
import pandas as pd


X_train = train_data_scaled[features]
y_train = train_data_scaled[target]
X_test = test_data_scaled[features]
y_test = test_data_scaled[target]

# Manually define three alpha (penalty) values for Ridge regression
alpha_values = [0.1, 1, 10]

# Store results for comparison
ridge_results = []

for alpha in alpha_values:
    # Fit Ridge with the specified alpha
    ridge = Ridge(alpha=alpha, random_state=42)
    ridge.fit(X_train, y_train)

    # Evaluate R-squared
    r2_train = ridge.score(X_train, y_train)
    r2_test = ridge.score(X_test, y_test)

    # Get coefficients
    coeffs = ridge.coef_

    # Store results
    ridge_results.append({
        "Alpha": alpha,
        "R-squared (Train)": r2_train,
        "R-squared (Test)": r2_test,
        "Non-zero Coefficients": sum(coeffs != 0)
    })

    # Print coefficients for each alpha
    coefficients = pd.DataFrame({
        "Feature": features,
        "Coefficient": coeffs
    })
    print(f"\nAlpha: {alpha}")
    print(coefficients.to_string(index=False))

# Convert results to DataFrame for easier comparison
ridge_results_df = pd.DataFrame(ridge_results)
print("\nSummary of Ridge Results for Different Alphas:")
print(ridge_results_df.to_string(index=False))

# Identify the best alpha based on test R-squared
best_alpha_ridge = ridge_results_df.loc[ridge_results_df['R-squared (Test)'].idxmax(), 'Alpha']

# Train Ridge with the best alpha
ridge_best = Ridge(alpha=best_alpha_ridge, random_state=42)
ridge_best.fit(X_train, y_train)

# Evaluate R-squared on test data
r2_train_best = ridge_best.score(X_train, y_train)
r2_test_best = ridge_best.score(X_test, y_test)

# Coefficients for the best Ridge model
coefficients_best = pd.DataFrame({
    "Feature": features,
    "Coefficient": ridge_best.coef_
})

# Output coefficients of the best model
print(f"\nBest alpha (penalty parameter): {best_alpha_ridge}")
print(f"R-squared (Training): {r2_train_best}")
print(f"R-squared (Test): {r2_test_best}")
print("\nCoefficients of the Best Ridge Model:")
print(coefficients_best.to_string(index=False))

# Identify non-zero coefficients
non_zero_coeffs_ridge = coefficients_best.query("Coefficient != 0")
print(f"\nNumber of non-zero coefficients: {len(non_zero_coeffs_ridge)}")
print(non_zero_coeffs_ridge)

# Random permutation importance for the best Ridge model
perm_importance_ridge = permutation_importance(ridge_best, X_test, y_test, n_repeats=10, random_state=42)
importance_df_ridge = pd.DataFrame({
    "Feature": features,
    "Importance": perm_importance_ridge.importances_mean
}).sort_values(by="Importance", ascending=False)

# Top 5 features by importance
top_5_features_ridge = importance_df_ridge.head(5)
print("\nTop 5 Most Important Features (Random Permutation Method - Ridge):")
print(top_5_features_ridge.to_string(index=False))



Alpha: 0.1
                Feature  Coefficient
               Nunits_x     4.520312
       SumDailyVisits_x    -1.798099
MedDailyVisitsPerUnit_x     4.099175
               Nunits_y    -1.745959
       SumDailyVisits_y    -3.628231
MedDailyVisitsPerUnit_y    -3.044181
       total_population    -0.479939
             median_age     1.045878
              BlueState     0.135144

Alpha: 1
                Feature  Coefficient
               Nunits_x     4.491963
       SumDailyVisits_x    -1.736120
MedDailyVisitsPerUnit_x     4.066844
               Nunits_y    -1.747639
       SumDailyVisits_y    -3.616947
MedDailyVisitsPerUnit_y    -3.028927
       total_population    -0.481324
             median_age     1.044122
              BlueState     0.134152

Alpha: 10
                Feature  Coefficient
               Nunits_x     4.253122
       SumDailyVisits_x    -1.221428
MedDailyVisitsPerUnit_x     3.786194
               Nunits_y    -1.770266
       SumDailyVisits_y    -3.491562
MedDa

Best Penalty Parameter (α):
The best penalty parameter is 1.0.

Reason: α = 1.0 achieves the highest R-squared on the test sample (0.698), which is comparable to the training R-squared (0.700) and demonstrates strong generalization. The model retains all 9 features with non-zero coefficients, striking a balance between regularization and predictive accuracy.
Non-Zero Coefficients (α = 1.0):
All 9 features in the dataset have non-zero coefficients:

Nunits_x (4.49)
SumDailyVisits_x (-1.74)
MedDailyVisitsPerUnit_x (4.07)
Nunits_y (-1.75)
SumDailyVisits_y (-3.62)
MedDailyVisitsPerUnit_y (-3.03)
total_population (-0.48)
median_age (1.04)
BlueState (0.13)
Signs and Magnitude Confirmation:
The large positive coefficients for Nunits_x and MedDailyVisitsPerUnit_x confirm prior expectations about their significance in explaining the response variable.
Negative coefficients for features like SumDailyVisits_y and MedDailyVisitsPerUnit_y are consistent with their expected inverse relationships.
The relatively small coefficients for total_population, median_age, and BlueState align with their lower anticipated importance.
R-Squared Comparison:
Ridge (α = 1.0): Test R-squared = 0.698
Linear Regression: Test R-squared = 0.698
LASSO (α = 0.01): Test R-squared = 0.667
The Ridge model performs equally well as linear regression in terms of R-squared, and better than the LASSO model. Ridge retains all features, making it more suited for datasets where all variables contribute meaningful information.

Feature Importance (Random Permutation Method):
The five most important features are:

Nunits_x (1.47)
MedDailyVisitsPerUnit_x (0.35)
SumDailyVisits_y (0.19)
Nunits_y (0.14)
median_age (0.07)
Comparison to LASSO:
Consistent Features: Nunits_x, MedDailyVisitsPerUnit_x, and SumDailyVisits_y appear as top features in both models.
Differences: Ridge places greater importance on Nunits_y and median_age, while LASSO emphasizes MedDailyVisitsPerUnit_y.
Conclusion:
The Ridge model at α = 1.0 performs well, matching the R-squared of linear regression while retaining all features. It identifies Nunits_x as the most critical predictor, consistent with prior knowledge and LASSO results. Ridge emphasizes smoother contributions from all features, making it useful when all predictors are believed to be relevant.